In [ ]:
! pip install git+git://github.com/huggingface/transformers/
!pip install --upgrade tensorflow
!python -m pip install --upgrade pip

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

# Pre-Processing  dataset and Training Model

In [ ]:
!mkdir wikiPlots
!unzip '/content/gdrive/MyDrive/WikiMoviePlots.zip' -d 'wikiPlots'
import pandas as pd
import io
import numpy as np

In [ ]:
df = pd.read_csv("/content/wikiPlots/wiki_movie_plots_deduped.csv", na_values = ['no info', '.'])
df.reset_index(drop=True, inplace=True)
genre_selection=df.loc[df['Genre'] == "comedy"]
tmp=genre_selection['Plot'].tolist()
tmp= np.array([('<BOS> <comedy> '+xi +' <EOS>') for xi in tmp])
np.savetxt(r'/content/wikiPlots/comedyText.txt',tmp, fmt='%s')

In [ ]:
!split -b $(expr $(cat '/content/wikiPlots/comedyText.txt' | wc -m) \* 80 / 100) '/content/wikiPlots/comedyText.txt' 'comedy_train.txt'

In [ ]:
!pip install transformers torch

In [ ]:
import logging
import math
import os
from dataclasses import dataclass, field
from typing import Optional

from transformers import (
    CONFIG_MAPPING,
    MODEL_WITH_LM_HEAD_MAPPING,
    AutoConfig,
    AutoModelWithLMHead,
    GPT2LMHeadModel,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    DataCollatorForPermutationLanguageModeling,
    HfArgumentParser,
    LineByLineTextDataset,
    PreTrainedTokenizer,
    TextDataset,
    Trainer,
    TrainingArguments,
    set_seed,
)

# Setup logging
logger = logging.getLogger(__name__)

# Get access to model types and model configs to select GPT2 model and config
MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)



In [ ]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune, or train from scratch.
    """

    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The model checkpoint for weights initialization. Leave None if you want to train a model from scratch."
        },

    )
    model_type: Optional[str] = field(
        default=None,
        metadata={
            "help": "If training from scratch, pass a model type from the list: "
            + ", ".join(MODEL_TYPES)
        },
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={
            "help": "Where do you want to store the pretrained models downloaded from s3"
        },
    )


In [ ]:
@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    train_data_file: Optional[str] = field(
        default=None, metadata={"help": "The input training data file (a text file)."}
    )
    eval_data_file: Optional[str] = field(
        default=None,
        metadata={
            "help": "An optional input evaluation data file to evaluate the perplexity on (a text file)."
        },
    )
    line_by_line: bool = field(
        default=False,
        metadata={
            "help": "Whether distinct lines of text in the dataset are to be handled as distinct sequences."
        },
    )

    mlm: bool = field(
        default=False,
        metadata={
            "help": "Train with masked-language modeling loss instead of language modeling."
        },
    )

    block_size: int = field(
        default=-1,
        metadata={
            "help": "Optional input sequence length after tokenization."
            "The training dataset will be truncated in block of this size for training."
            "Default to the model max input length for single sentence inputs (take into account special tokens)."
        },
    )
    overwrite_cache: bool = field(
        default=False,
        metadata={"help": "Overwrite the cached training and evaluation sets"},
    )



In [ ]:
# Create LineByLineDataset from Movie Plots text file
def get_dataset(
    args: DataTrainingArguments, tokenizer: PreTrainedTokenizer, evaluate=False
):
    file_path = args.eval_data_file if evaluate else args.train_data_file
    if args.line_by_line:
        return LineByLineTextDataset(
            tokenizer=tokenizer, file_path=file_path, block_size=args.block_size
        )
    else:
        return TextDataset(
            tokenizer=tokenizer,
            file_path=file_path,
            block_size=args.block_size,
            overwrite_cache=args.overwrite_cache,
        )


In [ ]:
def main():

    model_args = ModelArguments(
        model_name_or_path="/content/gdrive/MyDrive/story_generator_checkpoint", model_type="gpt2-medium"
        # , cache_dir=""
    )
    data_args = DataTrainingArguments(
        train_data_file="/content/train_text1.txt",
        eval_data_file="/content/eval_text1.txt",
        line_by_line=True,
        block_size=512,
        overwrite_cache=True,
    )
    training_args = TrainingArguments(
        output_dir="story_generator_checkpoint",
        overwrite_output_dir=True,
        do_train=True,
        do_eval=True,
        # evaluate_during_training=False,
        logging_steps=500,
        per_device_train_batch_size=2,
        num_train_epochs=2,
        save_total_limit=1,
        save_steps=1000,
        learning_rate=2e-5,
        per_device_eval_batch_size=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    )

    if data_args.eval_data_file is None and training_args.do_eval:
        raise ValueError(
            "Cannot do evaluation without an evaluation data file. Either supply a file to --eval_data_file "
            "or remove the --do_eval argument."
        )

    if (
        os.path.exists(training_args.output_dir)
        and os.listdir(training_args.output_dir)
        and training_args.do_train
        and not training_args.overwrite_output_dir
    ):
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
        )

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.local_rank,
        training_args.device,
        training_args.n_gpu,
        bool(training_args.local_rank != -1),
        training_args.fp16,
    )
    logger.info("Training/evaluation parameters %s", training_args)

    # Set seed for deterministic training runs
    set_seed(training_args.seed)


    config = AutoConfig.from_pretrained(
        model_args.model_name_or_path, cache_dir=model_args.cache_dir
    )

    tokenizer = AutoTokenizer.from_pretrained(
        model_args.model_name_or_path, cache_dir=model_args.cache_dir
    )

    model = GPT2LMHeadModel.from_pretrained(
        model_args.model_name_or_path,
        from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
    )

    special_tokens_dict = {
        "bos_token": "<BOS>",
        "eos_token": "<EOS>",
        "pad_token": "<PAD>",
        "additional_special_tokens": [
            "<superhero>",
            "<action>",
            "<drama>",
            "<thriller>",
            "<horror>",
            "<comedy>",
            "<sci_fi>"
        ],
    }

    num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
    model.resize_token_embeddings(len(tokenizer))

    if data_args.block_size <= 0: 
      # If block_size <= 0, set it to max. possible value allowed by model
        data_args.block_size = tokenizer.model_max_length
    else:
        data_args.block_size = min(data_args.block_size, tokenizer.model_max_length)

    # Get datasets

    train_dataset = (
        get_dataset(data_args, tokenizer=tokenizer) if training_args.do_train else None
    )
    eval_dataset = (
        get_dataset(data_args, tokenizer=tokenizer, evaluate=True)
        if training_args.do_eval
        else None
    )
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=data_args.mlm,
    )

    # Initialize our Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        # prediction_loss_only=True,
    )

   

In [ ]:
 # Training
    try:
      if training_args.do_train:
          model_path = (
              model_args.model_name_or_path
              if model_args.model_name_or_path is not None
              and os.path.isdir(model_args.model_name_or_path)
              else None
          )
          trainer.train(model_path=model_path)
          trainer.save_model()
          tokenizer.save_pretrained(training_args.output_dir)
    except KeyboardInterrupt:
      print("Saving model that was in the middle of training")
      trainer.save_model()
      tokenizer.save_pretrained(training_args.output_dir)
      return

    # Evaluation
    results = {}
    if training_args.do_eval:
        logger.info("*** Evaluate ***")

        eval_output = trainer.evaluate()

        perplexity = math.exp(eval_output["eval_loss"])
        result = {"perplexity": perplexity}

        output_eval_file = os.path.join(training_args.output_dir, "eval_results_lm.txt")
        if trainer.is_world_process_zero():
            with open(output_eval_file, "w") as writer:
                logger.info("***** Eval results *****")
                for key in sorted(result.keys()):
                    logger.info("  %s = %s", key, str(result[key]))
                    writer.write("%s = %s\n" % (key, str(result[key])))

        results.update(result)

    return results

if __name__ == "__main__":
    main()

In [ ]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()
torch.cuda.memory_summary(device=None, abbreviated=False)


## GPT2 Story Generator

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
# Run these cells for story generation
from transformers import pipeline, TextGenerationPipeline, GPT2LMHeadModel, AutoTokenizer

checkpoint = "/content/gdrive/MyDrive/story_generator_checkpoint"

model = GPT2LMHeadModel.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
story_generator = TextGenerationPipeline(model=model, tokenizer=tokenizer)
# The format for input_prompt: "<BOS> <genre> Optional text..."
# Supported genres: superhero, sci_fi, horror, thriller, action, drama , comedy

In [ ]:
input_prompt = "<BOS> <superhero> Superman "

genre="<BOS> <superhero>"
nextStr=genre+" "
input_list=["<BOS> <superhero> GreenArrow","Green Arrow will marry felicita"]
extra=100


for i in range(len(input_list)):
  if (i==0): 
    story = story_generator(input_list[i], max_length=300, do_sample=True,
                repetition_penalty=1.2, temperature=1.5, 
                top_p=0.95, top_k=50)
    lst_str = str(story)[38:-2]
    lst_str.replace('\\','')
    lst_str.replace(':','')
    lst_str.replace('/','')
    lst_str.replace('[citation needed]','')
    lst_str.replace('\\\'s',"'s")
    lst_str.replace("\\\\\\'s","'s")
    lst_str.replace("\\'s","'s")
    lst_str.replace("\'s","'s")
  else:
     inputPrmpt=genre+" "+lst_str+input_list[i]
     story = story_generator(inputPrmpt, max_length=300+extra, do_sample=True,
                repetition_penalty=1.2, temperature=1.5, 
                top_p=0.95, top_k=50)
     lst_str = str(story)[40:-3]
     lst_str.replace('/','')
     lst_str.replace('[citation needed]' and '\\\'s' ,'')
     lst_str.replace("\\\\\\'s","'s")
     lst_str.replace("\\'s","'s")
     lst_str.replace("\'s","'s")
     lst_str.replace("\'","'s")
     lst_str.replace("\\\'","")
     lst_str.replace("\\xa0all",'')
     lst_str.replace("xa0all",'')
     lst_str.replace("xa0w",'')
     lst_str.replace("\ ","")
     lst_str.replace("\\\'t","'t")


  # nextStr=genre+" "+lst_str
  
print(lst_str,sep='\n')


# story = story_generator(input_prompt, max_length=300, do_sample=True,
#                repetition_penalty=1.1, temperature=1.1, 
#                top_p=0.95, top_k=50)

# print(story)

In [ ]:
!pip install -U sentence-transformers
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
import re
splitted=re.split('; |, |\*|\n',lst_str)
splitted

In [ ]:
model_name='bert-base-nli-mean-tokens'
model=SentenceTransformer(model_name)
sentence_vec1=model.encode(splitted)
print(sentence_vec1)
encodedSentence1=[]
inputSentence1=["Superman wanted to save the people","Superman wanted to defeat lex","Superman realised he was not going to make it in time"]


In [ ]:

i=0
while i<len(inputSentence1):
  encodedSentence1.insert(i,model.encode(inputSentence1[i]))
  i+=1


i = 0
# k=0
list_of_probabilities1=[]
list_of_indicies1=[]
for z in range(len(inputSentence1)):
  while i < len(splitted):
    list_of_probabilities1.append(cosine_similarity([sentence_vec1[i]],[encodedSentence1[z]]))
    i += 1
  lf1 = [val for sublist in list_of_probabilities1 for val in sublist]
  lf3=[val for sublist in lf1 for val in sublist]
  print(lf3)
  x = max(lf3)
  list_of_indicies1.append(str(list_of_probabilities1.index(x)))
  print("Highest Probability equals::")
  print(x)
  print("Index::" + str(list_of_probabilities1.index(x)))
  print("Sentence::" + splitted[list_of_probabilities1.index(x)])
  #re initiallizing for next sentence
  list_of_probabilities1=[]
  i=0


[0.55044043, 0.59023726, 0.34525743, 0.29555026, 0.2212148, 0.51163626, 0.31914347, 0.56692296, 0.5472468, 0.33621794, 0.5374538]
Highest Probability equals::
0.59023726
Index::1
Sentence::Bizarro on his way to capture Lex Luthor alive and to claim his mind for a captive mind control formula that will create more powerful superheroes
[0.69780433, 0.7010479, 0.61354834, 0.36239588, 0.37286794, 0.6854222, 0.35786703, 0.7023071, 0.6634961, 0.33929905, 0.716362]
Highest Probability equals::
0.716362
Index::10
Sentence::unaware of Lexia)to attack him but are disarmed by Superman before they arrive to take Bruce hostage instead returning Wonderwoman to him. Luthor escapes however only realizing she may already have his back. Luthor leaves in tears with Batman calling Clark out for using
[0.4427575, 0.38232747, 0.33735064, 0.37573242, 0.28546485, 0.49798638, 0.46232992, 0.56747067, 0.47173285, 0.29046378, 0.5327936]
Highest Probability equals::
0.56747067
Index::7
Sentence::along they are met

In [ ]:
for i in range(len(inputSentence1)):
  splitted.insert(int(list_of_indicies1[i]),inputSentence1[i])

print(*splitted,sep='\n')


# EMBEDDING SENTENCES IN AN EXISTING STORY


In [ ]:
!pip install -U sentence-transformers
!pip install pyngrok
!pip install jupyter-dash

In [12]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyngrok import ngrok
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output,State

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import re

directory = '/content/drive/MyDrive/Colab Notebooks/KidsStoriesAnimals'

TempM=[]
Temp=[]
StryName=[]
# i=0
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        TempX=[]
        with open("/content/drive/MyDrive/Colab Notebooks/KidsStoriesAnimals"+'/'+filename,"r") as f:
           TempX = [line.replace('\n','') for line in f.readlines() ]
           TempX= [  line.split('.') for line in TempX]
           TempX= [val for sublist in TempX for val in sublist]
           TempX= [  line.split('?') for line in TempX]
           TempX= [val for sublist in TempX for val in sublist]
           TempX= list(filter(lambda x: x!=' ’ \n' and x!='"\n' and x!='\n' and x!='’\n' and x!=' \n', TempX))
          #  TempX= [ l for l in line.split('?') for line in TempX]
        TempM.append(TempX)
        StryName.append(filename)
        continue
    else:
      continue  
# TempM return each story as a list where each line is a string
print(TempM)           

In [ ]:
StryName

In [ ]:
StrpStry=[]
for i in range(0,4):
  with open("/content/drive/MyDrive/Colab Notebooks/KidsStoriesAnimals/"+StryName[i], "r") as a_file:
      x=a_file.readlines()
      StrpStry.append(x)
print(StrpStry)

TF-idf/Cosine Similarity

In [15]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
inputSentence=["The lion is known as the strongest animal.","The lion wieghts 40kg","Nearly all wild lions live in Africa.","Lions are the second largest big cat species in the world "] 

#Tf-idf part
ss=TempM[0]
TempMRemovedBraces = str(ss)[2:-2]

for j in range(len(StrpStry)):
  for i in range(len(inputSentence)):
 
    vectors=vectorizer.fit_transform([inputSentence[i],TempMRemovedBraces])
    # print(vectors)
    feature_names = vectorizer.get_feature_names()
    feature_names
    dense = vectors.todense()
    dense
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names)
    df.head()

In [ ]:
df.head()

In [ ]:
model_name='bert-base-nli-mean-tokens'
model=SentenceTransformer(model_name)

In [18]:
k=0
flattened=[]
while k<len(TempM):
  flattened.insert(k,TempM[k])

  k+=1

In [ ]:
# Flattened di gowaha el story based on el Tfidf l highest probabilistic story bas each line as a string
flattened

In [20]:
inputSentenceFind=["The lion is known as the strongest animal.","The lion wieghts 40kg","Nearly all wild lions live in Africa.","Lions are the second largest big cat species in the world "]



max_of_prob=[]

def iter(w):
  l=0
  i = 0
  list_of_probabilitiesFind=[]
  list_of_indiciesFind=[]
  list_of_summation=[]
  Sum=0
  encodedSentenceFind=[]


  while l<len(inputSentenceFind):
    encodedSentenceFind.insert(l,model.encode(inputSentenceFind[l]))
    l+=1

  sentence_vecFind=model.encode(flattened[w]) 
  for z in range(len(inputSentenceFind)):
    while i < len(flattened[w]):
      list_of_probabilitiesFind.append(cosine_similarity([sentence_vecFind[i]],[encodedSentenceFind[z]]))
      i += 1

      lfFind = [val for sublist in list_of_probabilitiesFind for val in sublist]
      lf2Find=[val for sublist in lfFind for val in sublist]
      Sum=sum(lf2Find)
    
    

  list_of_summation.insert(i,Sum)
  max_of_prob.append(Sum)
  print(list_of_summation)



for k in range(len(StryName)):
 iter(k)
  

[4.078954845666885]
[3.8967683725059032]
[5.183547884225845]
[3.651772966608405]
[3.969139814376831]
[3.762232281267643]
[5.480589911341667]
[4.690201431512833]
[5.670741245150566]
[3.1850018398836255]
[2.500084649771452]
[3.595407359302044]
[4.61608912050724]
[4.047555696219206]
[3.325271040201187]
[3.64690400660038]
[2.7894230894744396]
[3.7276982963085175]
[2.4197152256965637]
[2.9601545725017786]
[5.30040667951107]
[2.539410039782524]
[3.4446830600500107]
[9.089810458943248]
[8.332331269979477]
[7.58671922981739]
[4.29197883605957]
[8.50268267467618]
[7.438970921561122]
[11.702724866569042]
[1.297847032546997]
[3.6314667612314224]
[3.6071623638272285]
[3.358213111758232]
[3.0687973722815514]
[4.225390233099461]
[3.8077969513833523]
[3.8193444423377514]
[2.446199208498001]
[1.8353876173496246]
[3.6504612416028976]
[3.125282547902316]
[4.228650056757033]
[4.3569560796022415]
[9.909606277942657]
[4.80605760589242]
[2.506164714694023]
[2.6060285568237305]
[3.0842764638364315]
[2.531893

In [21]:
# Finding the matching story name and its index in our files
maximum=max(max_of_prob)
Indx=max_of_prob.index(maximum)
print("Index::",Indx)
print(StryName[Indx])

Index:: 29
the-camel,-the-jackal-and-the-crow.txt


## Cosine similarity to insert the sentences 


In [23]:
Story=TempM[Indx]
sentence_vec=model.encode(Story) 
encodedSentence=[]
inputSentence=["The lion is the strongest animal.","The lion wieghts 40kg","Nearly all wild lions live in Africa.","Lions are the second largest big cat species in the world "]
i=0
while i<len(inputSentence):
  encodedSentence.insert(i,model.encode(inputSentence[i]))
  i+=1


In [24]:
i = 0
# k=0
list_of_probabilities=[]
list_of_indicies=[]
for z in range(len(inputSentence)):
  while i < len(Story):
    list_of_probabilities.append(cosine_similarity([sentence_vec[i]],[encodedSentence[z]]))
    i += 1

  lf = [val for sublist in list_of_probabilities for val in sublist]
  lf2=[val for sublist in lf for val in sublist]
  x = max(lf2)
  list_of_indicies.append(str(list_of_probabilities.index(x)))
  print("Highest Probability equals::")
  print(x)
  print("Index::" + str(list_of_probabilities.index(x)))
  print("Sentence::" + Story[list_of_probabilities.index(x)])
  #re initiallizing for next sentence
  list_of_probabilities=[]
  i=0


Highest Probability equals::
0.565962
Index::11
Sentence::” asked the Lion
Highest Probability equals::
0.4437638
Index::30
Sentence::”The leopard intervened and said “Oh lord, I am bigger in size than my friends
Highest Probability equals::
0.69329476
Index::0
Sentence::Long ago in the dense forest of botswana, there lived a lion named wils
Highest Probability equals::
0.65163016
Index::30
Sentence::”The leopard intervened and said “Oh lord, I am bigger in size than my friends


In [25]:
for i in range(len(inputSentence)):
  Story.insert(int(list_of_indicies[i])+1,inputSentence[i])

print(*Story)


Long ago in the dense forest of botswana, there lived a lion named wils Nearly all wild lions live in Africa.  His loyal servants were a leopard, a jackal and a crow One day, the crow spotted a strange creature coming their way  He came flying back to his friends and said, “There is someone unusual coming this way  We should be careful ”“I am the king of the jungle,” said the lion  Why should I be afraid  Let’s go and meet him ”When they saw the animal they were all surprised  They had never seen such a creature with long legs and a big hump on its back  “Who is it and what is it doing in my jungle ” asked the Lion The lion is the strongest animal. “I am Todi, the camel,” he said  “I was a part of a trade caravan  I lost my way  Please help me!”Out of pity, the lion asked him to stay along with them  The other three were not happy, but they had to respect the Lion’s wishes Days passed by  One day the lion had a great fight with an elephant  The lion was badly injured and was too weak t

# Running with Flask

In [ ]:
!pip install flask_ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template

app=Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def 

# Running With JupyterDash

In [6]:
app=JupyterDash(__name__)
colors = {
    'background': '#100011',
    'text': '#465B9D'
}
app.layout=html.Div(#style={'backgroundColor': colors['background']}, children=[
    [
  html.H1("Hiding Learning Information",
          style={
            'textAlign': 'center',
            'color': colors['text'],
            'fontSize':'38px'

        }),
  html.Div(children='Please enter five facts about a specific animal',
    style={
        'fontSize':'18px',
        'fontFamily':'Copperplate',
        'fontWieght':'Bold',
        'textAlign': 'center',
    }),
  html.Div(dcc.Input(id='input-submit1', type='text',placeholder="Enter your First fact",style={'marginTop':'6px','width':'30%','border':'2px solid #aaa','borderRadius':'4px','padding':'6px','boxSizing':'border-box','borderColor':'dodgerBlue','boxShadow':'0 0 4px 0 dodgerBlue'}),style={'marginTop':'10px','textAlign':'center'}),   
  html.Div(dcc.Input(id='input-submit2', type='text',placeholder="Enter your Second fact",style={'marginTop':'6px','width':'30%','border':'2px solid #aaa','borderRadius':'4px','padding':'6px','boxSizing':'border-box','borderColor':'dodgerBlue','boxShadow':'0 0 4px 0 dodgerBlue'}),style={'marginTop':'10px','textAlign':'center'}),
  html.Div(dcc.Input(id='input-submit3', type='text',placeholder="Enter your Third fact",style={'marginTop':'6px','width':'30%','border':'2px solid #aaa','borderRadius':'4px','padding':'6px','boxSizing':'border-box','borderColor':'dodgerBlue','boxShadow':'0 0 4px 0 dodgerBlue'}),style={'marginTop':'10px','textAlign':'center'}),
  html.Div(dcc.Input(id='input-submit4', type='text',placeholder="Enter your Fourth fact",style={'marginTop':'6px','width':'30%','border':'2px solid #aaa','borderRadius':'4px','padding':'6px','boxSizing':'border-box','borderColor':'dodgerBlue','boxShadow':'0 0 4px 0 dodgerBlue'}),style={'marginTop':'10px','textAlign':'center'}),
  html.Div(dcc.Input(id='input-submit5', type='text',placeholder="Enter your Last fact",style={'marginTop':'6px','width':'30%','border':'2px solid #aaa','borderRadius':'4px','padding':'6px','boxSizing':'border-box','borderColor':'dodgerBlue','boxShadow':'0 0 4px 0 dodgerBlue'}),style={'marginTop':'10px','textAlign':'center'}),

    html.Div(html.Button('SUBMIT', id='submit-btn', n_clicks=0,style={
        'fontFamily':'Source Sans Pro sans-serif',
        'letterSpacing':'4px',
        'borderRadius':'20px',
        'outline':'none',
        'lineHeight':'1',
        'background-color':'#465B9D',
        'color': 'white',
        'padding': '12px' ,
        'marginTop':'25px',
        'textAlign': 'center',
        'display':'inline-block',
        'fontSize': '16px'
        }),
         style={'textAlign': 'center'})

])

# app.callback(
#     Output(component_id=),
#     [Input(component_id='input-submit1')],
#     prevent_initial_call=false
# )

In [ ]:
app.run_server(mode='external',debug=False)

In [ ]:
! ngrok authtoken 1vDOU1kP3NtXW5wm2fadH9gvk0c_47Pxg3adCqEeBx6x8Eoz2

In [ ]:
from pyngrok import ngrok
public_url=ngrok.connect('8050')

In [ ]:
public_url

In [ ]:
ngrok.kill()